## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate

In [ ]:
image = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
hist_original = cv2.calcHist([image], [0], None, [256], [0,256])

eq_image = cv2.equalizeHist(image)
hist_eq = cv2.calcHist([eq_image], [0], None, [256], [0,256])

lm = np.mean(image)

H1 = hist_original[:int(lm)+1]
H2 = hist_original[int(lm)+1:]

C1 = np.cumsum(H1) / np.max(np.cumsum(H1))
C2 = np.cumsum(H2) / np.max(np.cumsum(H2))

C1n = lm * C1
C2n = (lm + 1) + (255 - lm - 1) * C2

lut_bbhe = np.zeros((256,1))
lut_bbhe[:int(lm)+1, 0] = C1n
lut_bbhe[int(lm)+1:, 0] = C2n

result_image_bbhe = cv2.LUT(image, lut_bbhe.astype('uint8'))
hist_bbhe = cv2.calcHist([result_image_bbhe], [0], None, [256], [0,256])

plt.figure(figsize = (15, 10))
plt.subplot(3, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Obraz oryginalny')
plt.axis('off')

plt.subplot(3, 3, 2)
plt.imshow(eq_image, cmap='gray')
plt.title('Obraz po klasycznym HE')
plt.axis('off')

plt.subplot(3, 3, 3)
plt.imshow(result_image_bbhe, cmap='gray')
plt.title('Obraz po BBHE')
plt.axis('off')

plt.subplot(3, 3, 4)
plt.plot(hist_original)
plt.title('Oryginalny histogram')
plt.xlabel('Wartość piksela')
plt.ylabel('Liczba pikseli')

plt.subplot(3, 3, 5)
plt.plot(hist_eq)
plt.title('Histogram po klasycznym HE')
plt.xlabel('Wartość piksela')
plt.ylabel('Liczba pikseli')

plt.subplot(3, 3, 6)
plt.plot(hist_bbhe)
plt.title('Histogram po BBHE')
plt.xlabel('Wartość piksela')
plt.ylabel('Liczba pikseli')

plt.tight_layout()
plt.show()